In [2]:
from urllib.request import urlopen as uRequest
from bs4 import BeautifulSoup as soup
import re
import pandas as pd

page_url = 'https://www.thomann.de/it/ukulele_soprani.html'
uClient = uRequest(page_url)
html_page = uClient.read()
page_soup = soup(html_page)

records = []

containers = page_soup.findAll('div', {'class':'extensible-article'})
for container in containers:
    manufacturer  = container.findAll('span', {'class':'manufacturer'})[0].text #anche se la variabile ha un solo elemento dobbiamo comunque indicizzarlo
    model = container.findAll('span', {'class':'model'})[0].text
    image  = container.findAll('div',{'class':'product-image'})[0].img['src']
    price = container.findAll('span',{'class' : 'article-basketlink'})[0].text
    price = float(price.replace('€', '').replace(',','.'))
    reviews_no = container.findAll('span', {'class':'count'})[0].text
    availability = container.findAll('span', {'class':'rs-layover-trigger-text'})[0].text
    rating = container.findAll('span', {'class':'overlay-wrapper'})[0]['style']
    rating = float(re.search('[0-9]+(\.[0-9]+)?',rating).group())
    records.append([manufacturer, model, image, price, reviews_no, availability, rating])


data = pd.DataFrame(records, columns = ['manufacturer','model','image','price','reviews_no','availability','rating'])
data.head()

,manufacturer,model,image,price,reviews_no,availability,rating
0,Harley Benton,Ukulele UK-11DW Brown,https://thumbs.static-thomann.de/thumb/thumb15...,17.9,437,Disponibilità immediata,84.073
1,Harley Benton,Ukulele Pack,https://thumbs.static-thomann.de/thumb/thumb15...,30.0,20,Disponibilità immediata,88.000
2,Harley Benton,Hawaii Koa Soprano Ukulele,https://thumbs.static-thomann.de/thumb/thumb15...,99.0,13,Disponibilità immediata,93.846
3,Harley Benton,UK-12 Black,https://thumbs.static-thomann.de/thumb/thumb15...,19.9,357,Disponibilità immediata,78.543
4,Baton Rouge,V1-S nat,https://thumbs.static-thomann.de/thumb/thumb15...,49.0,0,Disponibilità immediata,0.000


# Domanda 2
[0-9]+ \,[0-9]+

# Domanda 3

rating = float(rating.replace('%', ''))

# Scaricare immagini

# 1 Creare directory

In [ ]:
import os

dest_dir = 'thaomann_img'
try:
    os.makedirs(dest_dir)
except:
    pass

# 2 Nome univoco all'immagine

In [ ]:
row = data.iloc[0]
ext = re.search('[^.]+$',row['image']).group() #cerchiamo l'estensione dell'immagine es .png
img_name = "img_{id:05d}.{ext:s}"
img_name.format(id=int(row.name), ext = ext)


# Percorso della cartella

In [ ]:
from os.path import join

path = join(dest_dir, img_name.format(id=int(row.name), ext = ext))
path

# Download dell'immagine

In [ ]:
from urllib.request import urlretrieve as retrieve

retrieve(row['image'], path)

In [ ]:
from PIL import Image 
Image.open(path)

In [ ]:
def download_images(data, dest_dir, img_name = "img_{id:05d}.{ext:s}"):
    data = data.copy()
    data['img_path'] = None
    img_paths = []
    for i in data.iterrows():
        ext = re.search('[^.]+$',row['image']).group() #cerchiamo l'estensione dell'immagine es .png
        path = join(dest_dir, img_name.format(id=int(row.name), ext = ext))
        retrieve(row['image'],path)
        img_paths.append(path)
    data['img_path'] = img_paths
    return data

In [ ]:
page_links=[c.a['href']for c in list(filter(lambda c:'active'not in c['class'],page_soup.findAll('div',{'class':'page'})))]
page_links

In [ ]:
base_url = re.search('http(s)?://[^/]*', page_url).group(0)
page_links = [base_url + l for l in page_links]

In [15]:
from urllib.request import urlopen as uRequest
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import os
from glob import glob
from os.path import join
from urllib.request import urlretrieve as retrieve

def scrape(page_url, records = []):
    uClient = uRequest(page_url)
    html_page = uClient.read()
    page_soup = soup(html_page)

    records = []

    containers = page_soup.findAll('div', {'class':'extensible-article'})
    for container in containers:
        manufacturer  = container.findAll('span', {'class':'manufacturer'})[0].text #anche se la variabile ha un solo elemento dobbiamo comunque indicizzarlo
        model = container.findAll('span', {'class':'model'})[0].text
        image  = container.findAll('div',{'class':'product-image'})[0].img['src']
        price = container.findAll('span',{'class' : 'article-basketlink'})[0].text
        price = float(price.replace('€', '').replace(',','.'))
        reviews_no = container.findAll('span', {'class':'count'})[0].text
        availability = container.findAll('span', {'class':'rs-layover-trigger-text'})[0].text
        rating = container.findAll('span', {'class':'overlay-wrapper'})[0]['style']
        rating = float(re.search('[0-9]+(\.[0-9]+)?',rating).group())
        records.append([manufacturer, model, image, price, reviews_no, availability, rating])
    return records

def navigate_scrape(f_page_url, records = []):
    page_html = uRequest(f_page_url).read()
    page_soup = soup(page_html)
    page_links = [c.a['href'] for c in \
                  list(filter(lambda c: 'active' not in c['class'], page_soup.findAll('div', {'class':'page'})))]
    base_url = re.search('http(s)?://[^/]*',f_page_url).group(0)
    page_links = [base_url + l for l in page_links]
    page_links = [f_page_url] + page_links
    
    for url in page_links:
        records += scrape(url, records)
    return records

def download_images(data, dest_dir, img_name = "img_{id:05d}.{ext:s}"):    
    data = data.copy()
    data['img_path'] = None
    img_paths = []
    for i, row in data.iterrows():
        ext = re.search('[^.]+$',row['image']).group() #cerchiamo l'estensione dell'immagine es .png
        path = join(dest_dir, img_name.format(id=int(row.name), ext = ext))
        retrieve(row['image'],path)
        img_paths.append(path)
    data['img_path'] = img_paths
    return data

page_url = 'https://www.thomann.de/it/ukulele_soprani.html'
records = navigate_scrape(page_url)
data = pd.DataFrame(records, columns=['manufacturer','model','image','price','review_no','availability','rating'])
data.info()
data.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 7 columns):
manufacturer    167 non-null object
model           167 non-null object
image           167 non-null object
price           167 non-null float64
review_no       167 non-null object
availability    167 non-null object
rating          167 non-null float64
dtypes: float64(2), object(5)
memory usage: 9.3+ KB


,manufacturer,model,image,price,review_no,availability,rating
162,Lanikai,Mahogany Soprano Ukule B-Stock,https://thumbs.static-thomann.de/thumb/thumb15...,93.0,0,Disponibilità immediata,0.0
163,Kala,Makala Soprano Ukulele B-Stock,https://thumbs.static-thomann.de/thumb/thumb15...,83.0,0,Disponibilità immediata,0.0
164,Islander,by Kanilea MS 4 HNS So B-Stock,https://thumbs.static-thomann.de/thumb/thumb15...,106.0,0,Disponibilità immediata,0.0
165,Ibanez,UKS10 B-Stock,https://thumbs.static-thomann.de/thumb/thumb15...,65.0,0,Disponibilità immediata,0.0
166,Kala,Ukadelic Soprano USA,https://thumbs.static-thomann.de/thumb/thumb15...,48.0,0,Disponibilità immediata,0.0
